In [1]:
import os

In [2]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project'

In [14]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/arsalanazhar2003/End-to-End-Gender-Classification-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="arsalanazhar2003"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "ddbdda8eff3c518a6b060fc3d7a9cd36bc8c292d"

In [21]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    metric_file_name: Path
    mlflow_uri: str

In [22]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        self.config = self.config.model_evaluation
        create_directories([self.config.root_dir])
        
        

        eval_config = EvaluationConfig(
            path_of_model=self.config.path_of_model,
            training_data=self.config.path_of_model,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            metric_file_name=self.config.metric_file_name,
            mlflow_uri="https://dagshub.com/arsalanazhar2003/End-to-End-Gender-Classification-project.mlflow"

        )
        return eval_config




In [24]:
import os
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [25]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def get_training_val_test_dataset(self):
        self.dataset = tf.keras.preprocessing.image_dataset_from_directory(
            "artifacts/data_preprocessing/DATASET",
            shuffle=True,
            image_size=(224, 224),
            batch_size=32
        )
        self.train_ds, self.val_ds, self.test_ds = self.get_dataset_partitions_tf(self.dataset)
        self.train_ds = self.train_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
        self.val_ds = self.val_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
        self.test_ds = self.test_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
       
    def get_dataset_partitions_tf(self, ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
        ds_size = sum(1 for _ in ds)  # Compute dataset size dynamically
        if shuffle:
            ds = ds.shuffle(shuffle_size, seed=12)
        train_size = int(train_split * ds_size)
        val_size = int(val_split * ds_size)
        train_ds = ds.take(train_size)
        val_ds = ds.skip(train_size).take(val_size)
        test_ds = ds.skip(train_size + val_size)
        
        return train_ds, val_ds, test_ds
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.get_training_val_test_dataset()
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            self.score = self.model.evaluate(self.test_ds)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("Loss", self.score[0])
            mlflow.log_metric("Accuracy", self.score[1])

            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(self.model, "model", registered_model_name="VGG16")
            else:
                mlflow.sklearn.log_model(self.model, "model")

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [26]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2024-09-27 06:18:48,529: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-27 06:18:48,532: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-27 06:18:48,532: INFO: common: created directory at: artifacts]
[2024-09-27 06:18:48,546: INFO: common: created directory at: artifacts/model_evaluation]
[2024-09-27 06:18:49,001: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2064 files belonging to 6 classes.
[2024-09-27 06:20:50,471: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /arsalanazhar2003/End-to-End-Gender-Classification-project.mlflow/api/2.0/mlflow/runs/create]
7/7 ━━━━━━━━━━━━━━━━━━━━ 84s 12s/step - accuracy: 0.2270 - loss: 9.3101


Registered model 'VGG16' already exists. Creating a new version of this model...
2024/09/27 06:22:34 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16, version 3
Created version '3' of model 'VGG16'.


[2024-09-27 06:22:34,828: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
